# Advanced Semantic Kernel - Multi-Agent Systems & Orchestration

Welcome to the advanced tutorial! This notebook builds on the basics to explore sophisticated multi-agent systems, orchestration patterns, and production-ready features.

## Prerequisites

Before starting this tutorial, make sure you've completed the **Semantic Kernel Basics** notebook. You should be familiar with:
- Basic agent creation and configuration
- Chat completion services
- Plugins and function calling
- Conversation threads

## What You'll Learn

- **Advanced Agent Features**: Streaming, reasoning models, structured outputs
- **Multi-Agent Orchestration**: Different patterns for agent collaboration
- **Human-in-the-Loop Systems**: Interactive agent workflows
- **Production Considerations**: Error handling, performance, best practices

## Resources
- [Agent Orchestration Guide](https://learn.microsoft.com/en-us/semantic-kernel/agents/orchestration/)
- [Multi-Agent Systems](https://learn.microsoft.com/en-us/semantic-kernel/agents/)
- [Structured Outputs](https://learn.microsoft.com/en-us/semantic-kernel/concepts/structured-outputs)

Let's dive into the advanced features!

## Setup and Imports

Let's start by importing all the necessary modules for advanced features.

In [1]:
import asyncio
import json
from typing import Annotated
import os
from dotenv import load_dotenv
from pydantic import BaseModel

# Core Semantic Kernel imports
from semantic_kernel import Kernel
from semantic_kernel.agents import Agent, ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings, AzureChatPromptExecutionSettings
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.contents import ChatMessageContent, TextContent, StreamingChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent

# Advanced orchestration imports
from semantic_kernel.agents import (
    GroupChatOrchestration, 
    RoundRobinGroupChatManager,
    ConcurrentOrchestration,
    SequentialOrchestration,
    HandoffOrchestration,
    OrchestrationHandoffs
)
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.agents.orchestration.tools import structured_outputs_transform
from semantic_kernel.agents.orchestration.group_chat import BooleanResult, GroupChatManager, MessageResult, StringResult
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings
from semantic_kernel.prompt_template import KernelPromptTemplate, PromptTemplateConfig
from typing_extensions import override

# Load environment variables
load_dotenv()

print("✅ Advanced imports loaded successfully!")

✅ Advanced imports loaded successfully!


In [2]:
# Configure the main chat completion service
chat_completion = AzureChatCompletion(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
)

# Configure reasoning model (if available)
reasoning_completion = None
if os.getenv("AZURE_REASONING_ENDPOINT"):
    reasoning_completion = AzureChatCompletion(
        api_key=os.getenv("AZURE_REASONING_API_KEY"),
        endpoint=os.getenv("AZURE_REASONING_ENDPOINT"),
        deployment_name=os.getenv("AZURE_REASONING_DEPLOYMENT_NAME"),
        instruction_role="developer",
    )
    print("✅ Reasoning model available!")

print("✅ Chat completion services configured!")

✅ Reasoning model available!
✅ Chat completion services configured!


## 1. Advanced Agent Features

Let's explore advanced agent capabilities including streaming responses and intermediate step handling.

### 1.1 Streaming Responses

Streaming allows you to see responses as they're generated, providing a better user experience for long responses.

In [4]:
# Create a simple agent for streaming demonstration
streaming_agent = ChatCompletionAgent(
    service=chat_completion,
    name="StreamingExpert",
    instructions="You are an expert who provides detailed explanations. Give comprehensive responses about topics.",
)

In [5]:
# Example 1: Regular (non-streaming) response
print("📝 Regular Response:")
print("=" * 40)
response = await streaming_agent.get_response(
    messages="Explain the benefits of renewable energy in detail."
)
print(response.message.content)

print("\n" + "=" * 40)

📝 Regular Response:
Here is a detailed overview of the key benefits associated with renewable energy:

1. Environmental Benefits  
   • Reduction of Greenhouse Gas Emissions  
     – Renewables such as wind, solar and hydroelectric produce little to no carbon dioxide (CO₂) or methane (CH₄) during operation, helping to slow climate change.  
     – Lifecycle emissions (from manufacturing to decommissioning) are far lower than those of coal, oil or natural gas.  
   • Air and Water Quality Improvement  
     – By displacing fossil-fuel combustion, renewables cut down on sulfur dioxide (SO₂), nitrogen oxides (NOₓ) and particulate emissions, leading to fewer smog-related illnesses.  
     – They consume far less water than thermal power plants, easing stress on rivers, lakes and aquifers.  
   • Biodiversity Preservation  
     – Well-planned renewable installations can minimize impacts on wildlife and habitats compared with mining, drilling and extensive infrastructure needed for fossil f

In [6]:
# Example 2: Streaming response using invoke_stream()
print("🌊 Streaming Response:")
print("=" * 40)
async for streaming_response in streaming_agent.invoke_stream(
    messages="Explain the challenges of implementing AI in healthcare."
):
    # This shows the response as it's being generated in real-time
    print(streaming_response.content, end="", flush=True)

print("\n\n✅ Streaming examples completed!")

# Create an agent with the analysis plugin for the next section
analysis_agent = ChatCompletionAgent(
    service=chat_completion,
    name="AnalysisExpert",
    instructions="You are an expert analyst. Use your tools to provide thorough analysis and recommendations.",
    plugins=[AnalysisPlugin()]
)

print("✅ Analysis agent created for next section!")

🌊 Streaming Response:
Implementing AI in healthcare holds great promise but also faces a range of interconnected challenges. Broadly, they can be grouped into technical, organizational, regulatory, ethical, and cultural categories:

1. Data Quality and Availability  
   • Fragmented systems: Medical records often reside in disparate systems (EHRs, lab databases, imaging archives) that don’t communicate easily.  
   • Incomplete or inconsistent data: Missing fields, variable coding practices, and unstructured notes hamper algorithm training.  
   • Labeling requirements: Supervised machine-learning models require expertly annotated data (e.g., images labeled by radiologists), which is time-consuming and expensive to produce.

2. Privacy, Security, and Compliance  
   • Patient confidentiality: AI solutions need to comply with laws such as HIPAA in the U.S., GDPR in Europe, and other local regulations, ensuring no unauthorized disclosure of protected health information.  
   • Cybersecur

### 1.2 Intermediate Steps with invoke()

**Understanding Agent Methods:**

- **`get_response()`**: Provides the final response only - blocks until the agent is completely done processing
- **`invoke()`**: Allows you to see the agent's step-by-step thinking process, including intermediate function calls and results
- **`invoke_stream()`**: Provides real-time streaming of the response as it's being generated

The `invoke()` method is particularly useful when you want to understand how the agent is working with plugins and function calls.

In [7]:
class AnalysisPlugin:
    """Plugin that provides analysis capabilities."""
    
    @kernel_function(description="Analyze a topic and provide insights.")
    def analyze_topic(self, topic: Annotated[str, "The topic to analyze"]) -> str:
        # Simulate analysis processing
        return f"Analysis of '{topic}': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility."
    
    @kernel_function(description="Generate recommendations based on analysis.")
    def generate_recommendations(self, analysis: Annotated[str, "The analysis to base recommendations on"]) -> str:
        return "Recommendations: 1) Conduct further research, 2) Develop a prototype, 3) Test with users, 4) Iterate based on feedback."


In [8]:
async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    """Handle intermediate steps in the agent's processing."""
    for item in message.items or []:
        if isinstance(item, FunctionResultContent):
            print(f"🔧 Function Result: {item.result}")
        elif isinstance(item, FunctionCallContent):
            print(f"📞 Function Call: {item.name} with arguments: {item.arguments}")
        else:
            print(f"💭 {message.name}: {message.content}")

# Test intermediate steps
task = "Analyze the potential of AI-powered educational tools and provide recommendations."
print(f"🎯 Task: {task}")
print("=" * 60)

async for response in analysis_agent.invoke(
    messages=task,
    on_intermediate_message=handle_intermediate_steps
):
    print(f"✅ Final Response from {response.name}: {response.content}")

🎯 Task: Analyze the potential of AI-powered educational tools and provide recommendations.
📞 Function Call: AnalysisPlugin-analyze_topic with arguments: {"topic":"AI-powered educational tools"}
🔧 Function Result: Analysis of 'AI-powered educational tools': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility.
📞 Function Call: AnalysisPlugin-generate_recommendations with arguments: {"analysis":"Analysis of 'AI-powered educational tools': This is a complex topic that requires consideration of market trends, user needs, and technical feasibility."}
🔧 Function Result: Recommendations: 1) Conduct further research, 2) Develop a prototype, 3) Test with users, 4) Iterate based on feedback.
✅ Final Response from AnalysisExpert: Analysis of AI-Powered Educational Tools

1. Market Trends  
   • Rapid Growth: The global edtech market has been expanding at over 15% annually, driven by increased remote and hybr

In [9]:
# Straming with intermediate steps
async for streaming_response in analysis_agent.invoke_stream(
    messages=task,
    on_intermediate_message=handle_intermediate_steps
):
    print(streaming_response.content, end="", flush=True)
    

📞 Function Call: AnalysisPlugin-analyze_topic with arguments: {"topic":"AI-powered educational tools"}
🔧 Function Result: Analysis of 'AI-powered educational tools': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility.
📞 Function Call: AnalysisPlugin-generate_recommendations with arguments: {"analysis":"Analysis of 'AI-powered educational tools': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility."}
🔧 Function Result: Recommendations: 1) Conduct further research, 2) Develop a prototype, 3) Test with users, 4) Iterate based on feedback.
Analysis of AI-powered educational tools:

1. Market Trends  
   - Rapid adoption: Schools, universities, and corporate training programs increasingly adopt AI for personalized learning, automated grading, and intelligent tutoring.  
   - Investment growth: Venture capital f

### 1.3 Structured Outputs with Pydantic

You can enforce structured responses using Pydantic models, ensuring consistent output format.

In [10]:
# Define structured output models
class Step(BaseModel):
    explanation: str
    output: str

class MathSolution(BaseModel):
    steps: list[Step]
    final_answer: str

# Create an agent with structured output
settings = AzureChatPromptExecutionSettings()
settings.response_format = MathSolution

math_agent = ChatCompletionAgent(
    service=chat_completion,
    name="MathTutor",
    instructions="You are a math tutor. Solve problems step by step and provide clear explanations.",
    arguments=KernelArguments(settings=settings)
)

# Test structured output
problem = "Solve: 3x + 7 = 22"
print(f"🧮 Problem: {problem}")

response = await math_agent.get_response(messages=problem)
solution = MathSolution.model_validate(json.loads(response.message.content))

print("\n📚 Structured Solution:")
print(solution.model_dump_json(indent=2))

🧮 Problem: Solve: 3x + 7 = 22

📚 Structured Solution:
{
  "steps": [
    {
      "explanation": "Subtract 7 from both sides of the equation 3x + 7 = 22 to isolate the term with x.",
      "output": "3x = 22 - 7 = 15"
    },
    {
      "explanation": "Divide both sides of the equation 3x = 15 by 3 to solve for x.",
      "output": "x = 15 ÷ 3 = 5"
    }
  ],
  "final_answer": "x = 5"
}


### 1.4 Reasoning Models (if available)

Some models like O1 provide enhanced reasoning capabilities. Let's test if we have access to reasoning models.

In [31]:
if reasoning_completion:
    print("🧠 Testing reasoning model...")
    
    reasoning_settings = AzureChatPromptExecutionSettings(
        service_id="reasoning",
        reasoning_effort="high"
    )
    
    reasoning_agent = ChatCompletionAgent(
        name="ReasoningExpert",
        instructions="You are an expert problem solver. Think through complex problems step by step.",
        service=reasoning_completion,
    )
    
    thread = ChatHistoryAgentThread()
    response = await reasoning_agent.get_response(
        messages="Solve: 2x + 3y = 7, 4x - y = 1. Show your reasoning.", 
        thread=thread
    )
    
    print(f"🧠 Reasoning Response: {response.message.content}")

    # Add another user message to the thread
    print("\n🤔 User: Is the solution correct? Explain.")
    response = await reasoning_agent.get_response(
        messages="Is the solution correct? Explain.", 
        thread=thread
    )
    print(f"🧠 Reasoning Response: {response.message.content}")
else:
    print("⚠️ Reasoning model not configured. Skipping reasoning example.")
    print("💡 To enable reasoning, configure AZURE_REASONING_* environment variables.")

🧠 Testing reasoning model...
🧠 Reasoning Response: Here is one clear way (elimination):

1. Write the system:
   (1) 2x + 3y = 7  
   (2) 4x –  y = 1

2. Multiply equation (2) by 3 to align the y-terms:
   3·(4x – y = 1) ⇒ 12x – 3y = 3

3. Add this to equation (1):
   (2x + 3y = 7)  
 + (12x – 3y = 3)  
 ----------------  
   14x        = 10  

4. Solve for x:
   14x = 10  
   x = 10/14 = 5/7

5. Substitute x = 5/7 back into, say, equation (2):
   4x – y = 1  
   4·(5/7) – y = 1  
   20/7 – y = 1  
   –y = 1 – 20/7 = 7/7 – 20/7 = –13/7  
   y = 13/7

Solution: x = 5/7, y = 13/7.

🤔 User: Is the solution correct? Explain.
🧠 Reasoning Response: Yes. To check, substitute x=5/7 and y=13/7 into both equations:

1. Check 2x + 3y = 7  
   2·(5/7) + 3·(13/7) = (10/7) + (39/7) = 49/7 = 7 ✔

2. Check 4x – y = 1  
   4·(5/7) – (13/7) = (20/7) – (13/7) = 7/7 = 1 ✔

Since both equations hold true, the solution x = 5/7, y = 13/7 is correct.


In [32]:
# You can see the reasoning tokens used by the agent
async for message in thread.get_messages():
    print(f"📝 Message from {message.role}: {message.content[:20]}...")
    if message.metadata.get("usage", None):
        print(f"    🔍 Usage: {message.metadata['usage'].model_dump()}")

📝 Message from AuthorRole.USER: Solve: 2x + 3y = 7, ...
📝 Message from AuthorRole.ASSISTANT: Here is one clear wa...
    🔍 Usage: {'prompt_tokens': 55, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'completion_tokens': 514, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}}
📝 Message from AuthorRole.USER: Is the solution corr...
📝 Message from AuthorRole.ASSISTANT: Yes. To check, subst...
    🔍 Usage: {'prompt_tokens': 340, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'completion_tokens': 295, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}}


In [33]:
await thread.delete()

## 2. Multi-Agent Orchestration Patterns

Now let's explore different ways to orchestrate multiple agents working together.

### 2.1 Group Chat with Round Robin

Round robin orchestration ensures agents take turns in a specific order.

In [34]:
def create_content_team() -> list[Agent]:
    """Create a team of content creation agents."""
    
    writer = ChatCompletionAgent(
        name="ContentWriter",
        description="A creative content writer.",
        instructions="You create engaging content. Focus on creativity and readability.",
        service=chat_completion,
    )
    
    editor = ChatCompletionAgent(
        name="ContentEditor",
        description="A content editor and reviewer.",
        instructions="You review and improve content. Focus on clarity, grammar, and flow.",
        service=chat_completion,
    )
    
    return [writer, editor]

def agent_response_callback(message: ChatMessageContent) -> None:
    """Display agent responses."""
    print(f"**{message.name}**")
    print(message.content)
    print("-" * 50)

# Create the group chat
content_agents = create_content_team()
group_chat = GroupChatOrchestration(
    members=content_agents,
    manager=RoundRobinGroupChatManager(max_rounds=3),
    agent_response_callback=agent_response_callback,
)

print("✅ Group chat orchestration created!")

✅ Group chat orchestration created!


In [35]:
# Test group chat orchestration
runtime = InProcessRuntime()
runtime.start()

print("🎯 Task: Create content for a sustainable living blog post")
print("=" * 60)

try:
    orchestration_result = await group_chat.invoke(
        task="Create a compelling blog post about sustainable living tips for busy professionals.",
        runtime=runtime,
    )
    
    final_result = await orchestration_result.get()
    print(f"\n**Final Result**")
    print(final_result)
    
finally:
    await runtime.stop_when_idle()

🎯 Task: Create content for a sustainable living blog post
**ContentWriter**
Title: Sustainable Living for Busy Professionals: 8 Simple Habits You Can Start Today

Introduction  
In the whirlwind of meetings, emails, and deadlines, sustainable living can feel like yet another to-do on an ever-growing list. But small, intentional changes to your daily routine not only lighten your environmental footprint—they can also sharpen your focus, boost your well-being, and even save you money. Here are eight practical, time-efficient sustainability tips for busy professionals who want to make a real difference without adding stress to their schedules.

1. Morning Routine Reset: Reusable Over Disposable  
• Keep a sleek, insulated travel mug in your bag, ready for coffee runs.  
• Swap single-use paper towels for a set of washable cloths by the sink.  
• Opt for a bamboo toothbrush and refillable toothpaste tablets—easy to toss in your toiletry bag and kind to the planet.

Why it works: Reusable i

### 2.2 Concurrent Orchestration

Concurrent orchestration allows multiple agents to work on the same task simultaneously.

In [36]:
def create_research_team() -> list[Agent]:
    """Create a team of research specialists."""
    
    tech_researcher = ChatCompletionAgent(
        name="TechResearcher",
        instructions="You are a technology research expert. Analyze questions from a technical perspective.",
        service=chat_completion,
    )
    
    market_researcher = ChatCompletionAgent(
        name="MarketResearcher",
        instructions="You are a market research expert. Analyze questions from a business and market perspective.",
        service=chat_completion,
    )
    
    return [tech_researcher, market_researcher]

# Create concurrent orchestration
research_agents = create_research_team()
concurrent_orchestration = ConcurrentOrchestration(members=research_agents)

print("✅ Concurrent orchestration created!")

✅ Concurrent orchestration created!


In [37]:
# Test concurrent orchestration
runtime = InProcessRuntime()
runtime.start()

print("🎯 Research Topic: Electric Vehicle Market")
print("=" * 50)

try:
    orchestration_result = await concurrent_orchestration.invoke(
        task="Analyze the current state and future prospects of the electric vehicle market.",
        runtime=runtime,
    )
    
    results = await orchestration_result.get(timeout=60)
    
    for result in results:
        print(f"**{result.name}**")
        print(result.content)
        print("-" * 50)
        
finally:
    await runtime.stop_when_idle()

🎯 Research Topic: Electric Vehicle Market
**TechResearcher**
Below is a structured, technical overview of the electric vehicle (EV) market’s current status and its future trajectory.

1. Market Overview  
   • Global sales and penetration  
     – In 2023, roughly 14 percent of new passenger-car sales worldwide were electric (battery EVs plus plug-in hybrids).  
     – Absolute EV sales surpassed 10 million units, with China accounting for about half, Europe about one-third, and North America around 15 percent.  
   • Leading manufacturers  
     – Tesla remains the largest single EV maker by volume; Volkswagen Group, BYD, Renault–Nissan–Mitsubishi Alliance and Hyundai–Kia follow closely.  
     – New entrants (e.g., Rivian, Lucid, NIO, XPeng) and legacy OEMs with dedicated EV platforms (e.g., GM’s Ultium, Ford’s MEB-derived models) are rapidly expanding.  
   • Regional dynamics  
     – China: aggressive subsidies, local champions, dense urban charging networks.  
     – Europe: stro

### 2.3 Concurrent Orchestration with Structured Outputs

You can combine concurrent orchestration with structured outputs for consistent results.

In [38]:
# Define structured output for research
class ResearchInsight(BaseModel):
    """Research insight with key findings."""
    key_finding: str
    supporting_evidence: str
    implications: str

class ResearchPerspectives(BaseModel):
    """Combined research perspectives."""
    technology: ResearchInsight
    market: ResearchInsight

# Create structured concurrent orchestration
structured_concurrent = ConcurrentOrchestration[str, ResearchPerspectives](
    members=research_agents,
    output_transform=structured_outputs_transform(ResearchPerspectives, chat_completion),
)

print("✅ Structured concurrent orchestration created!")

✅ Structured concurrent orchestration created!


In [39]:
# Test structured concurrent orchestration
runtime = InProcessRuntime()
runtime.start()

print("🎯 Structured Research: AI in Healthcare")
print("=" * 50)

try:
    orchestration_result = await structured_concurrent.invoke(
        task="Research AI applications in healthcare. Provide key finding, evidence, and implications.",
        runtime=runtime,
    )
    
    result = await orchestration_result.get(timeout=60)
    
    if isinstance(result, ResearchPerspectives):
        print("📊 Structured Research Results:")
        print(result.model_dump_json(indent=2))
    else:
        print(f"⚠️ Unexpected result type: {type(result)}")
        
finally:
    await runtime.stop_when_idle()

🎯 Structured Research: AI in Healthcare
📊 Structured Research Results:
{
  "technology": {
    "key_finding": "AI algorithms in healthcare demonstrate high performance in diagnostic imaging, drug discovery, predictive analytics, precision medicine, patient engagement, and administrative automation when integrated with clinical workflows and governance frameworks.",
    "supporting_evidence": "A 2020 Radiology meta-analysis reported AUCs of 0.94–0.97 for pneumonia detection on chest X-rays; Insilico Medicine identified a novel DDR1 kinase inhibitor in 46 days versus 18–24 months industry average; the TREWS early-warning system cut sepsis mortality by ~20%; DeepVariant called single nucleotide variants with >99% accuracy; a diabetes-management chatbot improved HbA1c by 0.5%; AI-assisted coding tools reduced claim denials by 30%.",
    "implications": "Enables early screening and workload reduction, accelerates R&D and lowers costs, supports proactive interventions and personalized therap

### 2.4 Sequential Orchestration

Sequential orchestration passes the output of one agent as input to the next, creating a pipeline.

In [52]:
def create_product_pipeline() -> list[Agent]:
    """Create a product development pipeline."""
    
    researcher = ChatCompletionAgent(
        name="ProductResearcher",
        instructions="You analyze product ideas and identify key features, target audience, and market opportunities.",
        service=chat_completion,
    )
    
    designer = ChatCompletionAgent(
        name="ProductDesigner",
        instructions="You create product concepts and specifications based on research insights.",
        service=chat_completion,
    )
    
    marketer = ChatCompletionAgent(
        name="ProductMarketer",
        instructions="You create marketing strategies and compelling messaging based on product designs.",
        service=chat_completion,
    )
    
    return [researcher, designer, marketer]

# Create sequential orchestration
product_pipeline = create_product_pipeline()
sequential_orchestration = SequentialOrchestration(
    members=product_pipeline
)

print("✅ Sequential orchestration created!")

✅ Sequential orchestration created!


In [53]:
# Test sequential orchestration
runtime = InProcessRuntime()
runtime.start()

print("🎯 Streaming Product Pipeline: Smart Home Security Device")
print("=" * 60)

try:
    orchestration_result = await sequential_orchestration.invoke(
        task="Develop a smart home security device that uses AI for threat detection.",
        runtime=runtime,
    )
    
    final_result = await orchestration_result.get(timeout=120)
    print(f"\n**Final Marketing Strategy**")
    print(final_result)
    
finally:
    await runtime.stop_when_idle()

🎯 Analysis Agent with Tools: Smart Home Security Device

**Final Marketing Strategy**
Below is a high-level go-to-market framework plus a suite of messaging assets you can adapt for sales collateral, digital ads and presentations. Feel free to pull what you need and let me know if you’d like deeper detail on any section.  

1. Target Segments & Positioning  
   • Facility & Safety Managers in high-rise offices, hotels, hospitals, manufacturing plants  
   • Construction sites and industrial plants (contractors, QHSE teams)  
   • Education, elder-care and residential-complex operators  
   • Premium consumer segment (outdoor enthusiasts, high-risk hobbyists)  

   Positioning Statement  
   “The world’s first wearable, multi-hazard evacuation device that cushions bodily impact, purifies life-saving air and automatically guides you to safety—even in zero-visibility emergencies.”  

2. Core Value Proposition and Messaging Pillars  
   1) Proactive Threat Response  
      – Instant hazard

### 2.5 Streaming and intermediate results with Orchestration

Lets looks at how you can use streaming and observe intermediate results with the sequential pattern. This will work for other orchestration patterns too.

In [56]:
# Global variable to track streaming state
is_new_message = True

def streaming_agent_response_callback(message: StreamingChatMessageContent, is_final: bool) -> None:
    """Handle streaming responses from agents."""
    global is_new_message
    
    if is_new_message:
        print(f"\n**{message.name}**")
        is_new_message = False
        
    print(message.content, end="", flush=True)
    
    if is_final:
        print("\n" + "-" * 50)
        is_new_message = True
        
def agent_response_callback(message: ChatMessageContent) -> None:
    """Handle support agent responses."""
    # print(f"**{message.name}**: {message.content}")
    
    # Show function calls
    for item in message.items:
        if isinstance(item, FunctionCallContent):
            print(f"  📞 Calling: {item.name}({item.arguments})")
        elif isinstance(item, FunctionResultContent):
            print(f"  📋 Result: {item.result}")
    print("-" * 50)


summarizer_agent = ChatCompletionAgent(
    service=chat_completion,
    name="Summarizer",
    instructions="You summarize results in no more than 100 words.",
)

# Create streaming sequential orchestration
streaming_sequential = SequentialOrchestration(
    members=[analysis_agent, summarizer_agent], ## Resusing reasoning agent for demonstration purposes.
    streaming_agent_response_callback=streaming_agent_response_callback,
    agent_response_callback=agent_response_callback
)

print("✅ Streaming sequential orchestration created!")

✅ Streaming sequential orchestration created!


In [57]:
# Test streaming sequential orchestration
runtime = InProcessRuntime()
runtime.start()

print("🎯 Analysis Agent with Tools: Fitness Wearable")
print("=" * 60)

try:
    orchestration_result = await streaming_sequential.invoke(
        task="Create a fitness wearable that tracks mental wellness alongside physical health.",
        runtime=runtime,
    )
    
    final_result = await orchestration_result.get(timeout=120)
    print(f"\n\n**Final Pipeline Result**")
    print(final_result)
    
finally:
    await runtime.stop_when_idle()

🎯 Analysis Agent with Tools: Fitness Wearable
  📞 Calling: AnalysisPlugin-analyze_topic({"topic":"Fitness wearable that tracks mental wellness alongside physical health"})
--------------------------------------------------

**AnalysisExpert**

--------------------------------------------------
  📋 Result: Analysis of 'Fitness wearable that tracks mental wellness alongside physical health': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility.
--------------------------------------------------

**AnalysisExpert**

--------------------------------------------------

**AnalysisExpert**
  📞 Calling: AnalysisPlugin-generate_recommendations({"analysis":"Analysis of 'Fitness wearable that tracks mental wellness alongside physical health': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility."})
---------------------

## 3. Custom Group Chat Manager

For more sophisticated group interactions, you can create custom group chat managers that use AI to decide who speaks next.

In [58]:
class AIGroupChatManager(GroupChatManager):
    """AI-powered group chat manager that decides who speaks next."""
    
    def __init__(self, topic: str, service, **kwargs):
        super().__init__(**kwargs)
        self.topic = topic
        self.service = service
        self.max_rounds = kwargs.get('max_rounds', 6)
        self.current_round = 0
    
    async def _render_prompt(self, prompt: str, arguments: KernelArguments) -> str:
        """Helper to render prompts with arguments."""
        config = PromptTemplateConfig(template=prompt)
        template = KernelPromptTemplate(prompt_template_config=config)
        return await template.render(Kernel(), arguments=arguments)
    
    @override
    async def should_request_user_input(self, chat_history: ChatHistory) -> BooleanResult:
        """This manager doesn't require user input."""
        return BooleanResult(result=False, reason="No user input required.")
    
    @override
    async def should_terminate(self, chat_history: ChatHistory) -> BooleanResult:
        """Determine if the discussion should end."""
        self.current_round += 1
        
        if self.current_round >= self.max_rounds:
            return BooleanResult(result=True, reason="Maximum rounds reached.")
        
        # Simple termination based on round count
        return BooleanResult(result=False, reason="Discussion continues.")
    
    @override
    async def select_next_agent(
        self, chat_history: ChatHistory, participant_descriptions: dict[str, str]
    ) -> StringResult:
        """Select the next agent to speak."""
        # Simple round-robin selection for this example
        agent_names = list(participant_descriptions.keys())
        selected_agent = agent_names[self.current_round % len(agent_names)]
        
        return StringResult(
            result=selected_agent,
            reason=f"Selected {selected_agent} for round {self.current_round}"
        )
    
    @override
    async def filter_results(self, chat_history: ChatHistory) -> MessageResult:
        """Filter and summarize the discussion results."""
        summary = f"Discussion on '{self.topic}' completed after {self.current_round} rounds."
        
        return MessageResult(
            result=ChatMessageContent(role=AuthorRole.ASSISTANT, content=summary),
            reason="Discussion summarized"
        )

print("✅ Custom AI Group Chat Manager created!")

✅ Custom AI Group Chat Manager created!


In [59]:
# Create debate agents
def create_debate_agents() -> list[Agent]:
    """Create agents for a debate scenario."""
    
    optimist = ChatCompletionAgent(
        name="TechOptimist",
        description="A technology optimist who believes in AI's positive potential.",
        instructions="You believe AI will greatly benefit humanity. Present positive arguments with specific examples.",
        service=chat_completion,
    )
    
    skeptic = ChatCompletionAgent(
        name="TechSkeptic",
        description="A technology skeptic who is concerned about AI risks.",
        instructions="You are concerned about AI risks and challenges. Present thoughtful counterarguments.",
        service=chat_completion,
    )
    
    return [optimist, skeptic]

# Create custom group chat
debate_agents = create_debate_agents()
custom_group_chat = GroupChatOrchestration(
    members=debate_agents,
    manager=AIGroupChatManager(
        topic="The Future of AI in Society",
        service=chat_completion,
        max_rounds=4,
    ),
    agent_response_callback=agent_response_callback,
)

print("✅ Custom group chat for debate created!")

ValidationError: 1 validation error for AIGroupChatManager
topic
  Object has no attribute 'topic' [type=no_such_attribute, input_value='The Future of AI in Society', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/no_such_attribute

In [ ]:
# Test custom group chat
runtime = InProcessRuntime()
runtime.start()

print("🎯 AI Debate: The Future of AI in Society")
print("=" * 60)

try:
    orchestration_result = await custom_group_chat.invoke(
        task="Discuss the benefits and risks of AI integration in society.",
        runtime=runtime,
    )
    
    final_result = await orchestration_result.get()
    print(f"\n**Debate Summary**")
    print(final_result)
    
finally:
    await runtime.stop_when_idle()

## 4. Handoff Orchestration

Handoff orchestration allows agents to transfer conversations to specialized agents based on the context.

In [ ]:
# Create specialized support plugins
class TechnicalSupportPlugin:
    """Plugin for technical support tasks."""
    
    @kernel_function(description="Diagnose technical issues.")
    def diagnose_issue(self, issue: Annotated[str, "Description of the technical issue"]) -> str:
        return f"Diagnosis for '{issue}': This appears to be a common connectivity issue. Try restarting your device and checking your network connection."
    
    @kernel_function(description="Provide troubleshooting steps.")
    def troubleshoot(self, diagnosis: Annotated[str, "The diagnosis result"]) -> str:
        return "Troubleshooting steps: 1) Restart device, 2) Check network, 3) Update software, 4) Contact support if issue persists."

class BillingSupportPlugin:
    """Plugin for billing support tasks."""
    
    @kernel_function(description="Check billing information.")
    def check_billing(self, account_id: Annotated[str, "Account ID"]) -> str:
        return f"Billing info for account {account_id}: Current balance: $45.99, Next payment due: 2024-02-15"
    
    @kernel_function(description="Process payment.")
    def process_payment(self, amount: Annotated[str, "Payment amount"]) -> str:
        return f"Payment of {amount} processed successfully. Thank you!"

print("✅ Support plugins created!")

In [ ]:
# Create specialized support agents
triage_agent = ChatCompletionAgent(
    name="SupportTriage",
    description="A triage agent that routes customers to appropriate specialists.",
    instructions="You help customers by understanding their needs and routing them to the right specialist. Be friendly and efficient.",
    service=chat_completion,
)

tech_support_agent = ChatCompletionAgent(
    name="TechSupport",
    description="A technical support specialist.",
    instructions="You provide technical support and troubleshooting. Use your tools to diagnose and solve problems.",
    service=chat_completion,
    plugins=[TechnicalSupportPlugin()],
)

billing_support_agent = ChatCompletionAgent(
    name="BillingSupport",
    description="A billing support specialist.",
    instructions="You handle billing questions and payment processing. Use your tools to help with account issues.",
    service=chat_completion,
    plugins=[BillingSupportPlugin()],
)

print("✅ Support agents created!")

In [ ]:
# Define handoff relationships
handoffs = (
    OrchestrationHandoffs()
    .add_many(
        source_agent=triage_agent.name,
        target_agents={
            tech_support_agent.name: "Transfer to this agent for technical issues and troubleshooting",
            billing_support_agent.name: "Transfer to this agent for billing, payment, and account issues",
        },
    )
    .add(
        source_agent=tech_support_agent.name,
        target_agent=triage_agent.name,
        description="Transfer back to triage if the issue is not technical",
    )
    .add(
        source_agent=billing_support_agent.name,
        target_agent=triage_agent.name,
        description="Transfer back to triage if the issue is not billing-related",
    )
)

print("✅ Handoff relationships defined!")

In [ ]:
# Create handoff orchestration
def support_response_callback(message: ChatMessageContent) -> None:
    """Handle support agent responses."""
    print(f"**{message.name}**: {message.content}")
    
    # Show function calls
    for item in message.items:
        if isinstance(item, FunctionCallContent):
            print(f"  📞 Calling: {item.name}({item.arguments})")
        elif isinstance(item, FunctionResultContent):
            print(f"  📋 Result: {item.result}")
    print("-" * 50)

handoff_orchestration = HandoffOrchestration(
    members=[
        triage_agent,
        tech_support_agent,
        billing_support_agent,
    ],
    handoffs=handoffs,
    agent_response_callback=support_response_callback,
)

print("✅ Handoff orchestration created!")

In [ ]:
# Test handoff orchestration
runtime = InProcessRuntime()
runtime.start()

print("🎯 Customer Support Scenario: Technical Issue")
print("=" * 60)

try:
    orchestration_result = await handoff_orchestration.invoke(
        task="Hi, I'm having trouble connecting to the internet. My device keeps disconnecting.",
        runtime=runtime,
    )
    
    final_result = await orchestration_result.get()
    print(f"\n**Support Resolution**")
    print(final_result)
    
finally:
    await runtime.stop_when_idle()

## 5. Human-in-the-Loop Systems

Sometimes you need human input during agent interactions. Here's how to implement human-in-the-loop systems.

In [ ]:
# Custom manager that requests human input
class HumanInTheLoopManager(RoundRobinGroupChatManager):
    """Manager that requests human input after specific conditions."""
    
    @override
    async def should_request_user_input(self, chat_history: ChatHistory) -> BooleanResult:
        """Request user input after every 2 agent exchanges."""
        agent_messages = [msg for msg in chat_history.messages if msg.role == AuthorRole.ASSISTANT]
        
        if len(agent_messages) >= 2 and len(agent_messages) % 2 == 0:
            return BooleanResult(
                result=True,
                reason="Requesting user feedback after agent exchanges"
            )
        
        return BooleanResult(
            result=False,
            reason="No user input needed yet"
        )

# Simple human response function for demonstration
def get_human_response(chat_history: ChatHistory) -> ChatMessageContent:
    """Get human input (simulated for this example)."""
    # In a real application, this would get actual user input
    responses = [
        "That's interesting! Can you elaborate on the benefits?",
        "I like the first approach better. Can you refine it?",
        "Good work! Please finalize the solution."
    ]
    
    # Simulate human response based on conversation length
    response_index = min(len(chat_history.messages) // 4, len(responses) - 1)
    human_input = responses[response_index]
    
    print(f"👤 Human: {human_input}")
    return ChatMessageContent(role=AuthorRole.USER, content=human_input)

print("✅ Human-in-the-loop components created!")

In [ ]:
# Create collaborative agents
def create_collaborative_agents() -> list[Agent]:
    """Create agents that work well with human feedback."""
    
    strategist = ChatCompletionAgent(
        name="Strategist",
        description="A strategic planner.",
        instructions="You develop high-level strategies and adapt based on feedback. Be responsive to human input.",
        service=chat_completion,
    )
    
    implementer = ChatCompletionAgent(
        name="Implementer",
        description="A solution implementer.",
        instructions="You create detailed implementation plans based on strategies. Incorporate human feedback.",
        service=chat_completion,
    )
    
    return [strategist, implementer]

# Create human-in-the-loop orchestration
collaborative_agents = create_collaborative_agents()
human_loop_orchestration = GroupChatOrchestration(
    members=collaborative_agents,
    manager=HumanInTheLoopManager(
        max_rounds=6,
        human_response_function=get_human_response,
    ),
    agent_response_callback=agent_response_callback,
)

print("✅ Human-in-the-loop orchestration created!")

In [ ]:
# Test human-in-the-loop system
runtime = InProcessRuntime()
runtime.start()

print("🎯 Collaborative Project: Remote Team Productivity")
print("=" * 60)

try:
    orchestration_result = await human_loop_orchestration.invoke(
        task="Develop a strategy to improve remote team productivity and collaboration.",
        runtime=runtime,
    )
    
    final_result = await orchestration_result.get()
    print(f"\n**Final Collaborative Solution**")
    print(final_result)
    
finally:
    await runtime.stop_when_idle()

## 6. Production Considerations

When building production systems, consider these important aspects:

### 6.1 Error Handling

Implement robust error handling for production systems.

In [ ]:
import logging
from typing import Optional

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class RobustAgent:
    """Agent wrapper with error handling and retries."""
    
    def __init__(self, agent: ChatCompletionAgent, max_retries: int = 3):
        self.agent = agent
        self.max_retries = max_retries
    
    async def get_response_with_retry(self, message: str, thread: Optional[ChatHistoryAgentThread] = None) -> Optional[str]:
        """Get response with retry logic and error handling."""
        
        for attempt in range(self.max_retries):
            try:
                logger.info(f"Attempt {attempt + 1} for agent {self.agent.name}")
                
                response = await self.agent.get_response(messages=message, thread=thread)
                logger.info(f"Success: {self.agent.name} responded")
                
                return response.message.content
                
            except Exception as e:
                logger.error(f"Attempt {attempt + 1} failed for {self.agent.name}: {str(e)}")
                
                if attempt == self.max_retries - 1:
                    logger.error(f"All retries exhausted for {self.agent.name}")
                    return None
                
                # Wait before retry
                await asyncio.sleep(2 ** attempt)  # Exponential backoff
        
        return None

# Test error handling
test_agent = ChatCompletionAgent(
    service=chat_completion,
    name="TestAgent",
    instructions="You are a test agent for demonstrating error handling."
)

robust_agent = RobustAgent(test_agent)
response = await robust_agent.get_response_with_retry("Hello, how are you?")

if response:
    print(f"✅ Robust response: {response}")
else:
    print("❌ Failed to get response after retries")

print("✅ Error handling example completed!")

### 6.2 Performance Monitoring

Monitor performance metrics for production systems.

In [ ]:
import time
from dataclasses import dataclass
from typing import Dict, List

@dataclass
class PerformanceMetrics:
    """Performance metrics for agent operations."""
    agent_name: str
    operation: str
    start_time: float
    end_time: float
    duration: float
    success: bool
    error_message: Optional[str] = None

class PerformanceMonitor:
    """Monitor and track agent performance."""
    
    def __init__(self):
        self.metrics: List[PerformanceMetrics] = []
    
    async def monitor_agent_call(self, agent: ChatCompletionAgent, message: str) -> Optional[str]:
        """Monitor an agent call and collect metrics."""
        start_time = time.time()
        
        try:
            response = await agent.get_response(messages=message)
            end_time = time.time()
            
            # Record successful metrics
            metrics = PerformanceMetrics(
                agent_name=agent.name,
                operation="get_response",
                start_time=start_time,
                end_time=end_time,
                duration=end_time - start_time,
                success=True
            )
            self.metrics.append(metrics)
            
            return response.message.content
            
        except Exception as e:
            end_time = time.time()
            
            # Record failed metrics
            metrics = PerformanceMetrics(
                agent_name=agent.name,
                operation="get_response",
                start_time=start_time,
                end_time=end_time,
                duration=end_time - start_time,
                success=False,
                error_message=str(e)
            )
            self.metrics.append(metrics)
            
            return None
    
    def get_performance_summary(self) -> Dict[str, any]:
        """Get performance summary statistics."""
        if not self.metrics:
            return {"message": "No metrics collected"}
        
        total_calls = len(self.metrics)
        successful_calls = sum(1 for m in self.metrics if m.success)
        failed_calls = total_calls - successful_calls
        
        durations = [m.duration for m in self.metrics if m.success]
        avg_duration = sum(durations) / len(durations) if durations else 0
        
        return {
            "total_calls": total_calls,
            "successful_calls": successful_calls,
            "failed_calls": failed_calls,
            "success_rate": successful_calls / total_calls if total_calls > 0 else 0,
            "average_duration": avg_duration,
            "max_duration": max(durations) if durations else 0,
            "min_duration": min(durations) if durations else 0,
        }

# Test performance monitoring
monitor = PerformanceMonitor()
test_agent = ChatCompletionAgent(
    service=chat_completion,
    name="PerformanceTestAgent",
    instructions="You are a test agent for performance monitoring."
)

# Make several monitored calls
test_messages = [
    "Hello, how are you?",
    "What's the weather like?",
    "Tell me a joke.",
]

print("📊 Testing performance monitoring...")
for i, message in enumerate(test_messages):
    print(f"Test {i+1}: {message}")
    response = await monitor.monitor_agent_call(test_agent, message)
    if response:
        print(f"Response: {response[:50]}...")
    print("-" * 30)

# Show performance summary
summary = monitor.get_performance_summary()
print("\n📈 Performance Summary:")
for key, value in summary.items():
    print(f"  {key}: {value}")

print("\n✅ Performance monitoring example completed!")

### 6.3 Best Practices Summary

Here are key best practices for production Semantic Kernel applications:

#### 🔧 Configuration Management
- Use environment variables for sensitive configuration
- Implement configuration validation
- Support multiple environments (dev, staging, production)

#### 🔍 Monitoring & Logging
- Implement comprehensive logging for debugging
- Monitor API usage and costs
- Track agent performance metrics
- Set up alerting for failures

#### 🛡️ Security
- Never log sensitive information
- Implement proper authentication and authorization
- Use secure communication channels
- Validate all inputs and outputs

#### 🚀 Performance
- Implement caching where appropriate
- Use concurrent orchestration for parallel tasks
- Monitor and optimize token usage
- Implement proper timeout handling

#### 🔄 Reliability
- Implement retry logic with exponential backoff
- Handle rate limiting gracefully
- Provide fallback mechanisms
- Clean up resources (threads, connections)

#### 📊 Testing
- Test agent responses with various inputs
- Implement integration tests for orchestration
- Test error scenarios and edge cases
- Monitor agent behavior in production

## 7. Cleanup

Always clean up resources when done.

In [ ]:
# Clean up any remaining threads and resources
print("🧹 Cleaning up resources...")

# Note: In a real application, you'd want to track and clean up all threads
# This is a simplified example

print("✅ Cleanup completed!")

## 🎉 Congratulations!

You've completed the Advanced Semantic Kernel tutorial! Here's what you've mastered:

### ✅ Advanced Concepts Covered:
1. **Streaming & Intermediate Steps** - Real-time response handling
2. **Structured Outputs** - Consistent response formatting with Pydantic
3. **Multi-Agent Orchestration** - Complex agent collaboration patterns
4. **Group Chat Management** - AI-powered conversation coordination
5. **Concurrent Processing** - Parallel agent execution
6. **Sequential Workflows** - Pipeline-based processing
7. **Handoff Systems** - Specialized agent routing
8. **Human-in-the-Loop** - Interactive agent workflows
9. **Production Considerations** - Error handling, monitoring, best practices

### 🚀 What's Next?
- Build your own multi-agent system for your specific use case
- Explore more advanced plugins and integrations
- Implement production monitoring and alerting
- Experiment with different AI models and configurations
- Contribute to the Semantic Kernel community

### 📚 Additional Resources:
- [Advanced Agent Patterns](https://learn.microsoft.com/en-us/semantic-kernel/agents/patterns/)
- [Production Deployment Guide](https://learn.microsoft.com/en-us/semantic-kernel/deployment/)
- [Semantic Kernel Community](https://github.com/microsoft/semantic-kernel/discussions)
- [Plugin Development Guide](https://learn.microsoft.com/en-us/semantic-kernel/agents/plugins/)
- [Performance Optimization](https://learn.microsoft.com/en-us/semantic-kernel/concepts/performance/)

### 🤝 Community
- Join the [Semantic Kernel Discord](https://discord.gg/semantic-kernel)
- Follow [@SemanticKernel](https://twitter.com/SemanticKernel) on Twitter
- Check out the [Developer Blog](https://devblogs.microsoft.com/semantic-kernel/)

Happy building with Semantic Kernel! 🚀🤖